In [ ]:
import os
from time import sleep

import numpy as np
import pandas as pd
import requests
from dotenv import load_dotenv

load_dotenv()

True

In [37]:
AV1 = os.getenv("AV1")
AV2 = os.getenv("AV2")
AV3 = os.getenv("AV3")
AV4 = os.getenv("AV4")

df = pd.read_csv("./predictions.csv")
tickers = df["ticker"].unique()

In [38]:
headers = {
    "accept": "application/json",
    "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36",
}

In [39]:
def get_fiscal_quarter(date_str):
    """Convert date string to fiscal quarter format (YYYYqQ)."""
    date = pd.to_datetime(date_str).to_period("Q")
    return date.strftime("%Yq%q")

## Get BTC prices


In [40]:
url = "https://api.blockchain.info/charts/market-price"
params = {"timespan": "all", "format": "json", "sampled": "true"}

r = requests.get(url, params=params, headers=headers, timeout=30)
r.raise_for_status()
data = r.json()

print(data)

btc_dict = {}
for value in data["values"]:
    x = value["x"]
    y = value["y"]
    date = pd.to_datetime(int(x), unit="s").to_period("Q")
    quarter = date.strftime("%Yq%q")
    btc_dict[quarter] = float(y)

{'status': 'ok', 'name': 'Market Price (USD)', 'unit': 'USD', 'period': 'day', 'description': 'Average USD market price across major bitcoin exchanges.', 'values': [{'x': 1230940800, 'y': 0.0}, {'x': 1231286400, 'y': 0.0}, {'x': 1231632000, 'y': 0.0}, {'x': 1231977600, 'y': 0.0}, {'x': 1232323200, 'y': 0.0}, {'x': 1232668800, 'y': 0.0}, {'x': 1233014400, 'y': 0.0}, {'x': 1233360000, 'y': 0.0}, {'x': 1233705600, 'y': 0.0}, {'x': 1234051200, 'y': 0.0}, {'x': 1234396800, 'y': 0.0}, {'x': 1234742400, 'y': 0.0}, {'x': 1235088000, 'y': 0.0}, {'x': 1235433600, 'y': 0.0}, {'x': 1235779200, 'y': 0.0}, {'x': 1236124800, 'y': 0.0}, {'x': 1236470400, 'y': 0.0}, {'x': 1236816000, 'y': 0.0}, {'x': 1237161600, 'y': 0.0}, {'x': 1237507200, 'y': 0.0}, {'x': 1237852800, 'y': 0.0}, {'x': 1238198400, 'y': 0.0}, {'x': 1238544000, 'y': 0.0}, {'x': 1238889600, 'y': 0.0}, {'x': 1239235200, 'y': 0.0}, {'x': 1239580800, 'y': 0.0}, {'x': 1239926400, 'y': 0.0}, {'x': 1240272000, 'y': 0.0}, {'x': 1240617600, 'y': 

## Get Hashrate


In [41]:
url = "https://api.blockchain.info/charts/hash-rate"
params = {"timespan": "all", "format": "json", "sampled": "true"}

r = requests.get(url, params=params, headers=headers, timeout=30)
r.raise_for_status()
data = r.json()

print(data)

hash_dict = {}
for value in data["values"]:
    x = value["x"]
    y = value["y"]
    date = pd.to_datetime(int(x), unit="s").to_period("Q")
    quarter = date.strftime("%Yq%q")
    hash_dict[quarter] = float(y)

{'status': 'ok', 'name': 'Hash Rate', 'unit': 'Hash Rate TH/s', 'period': 'day', 'description': 'The estimated number of tera hashes per second (trillions of hashes per second) the Bitcoin network is performing.', 'values': [{'x': 1230940800, 'y': 4.97102696296296e-08}, {'x': 1231286400, 'y': 0.0}, {'x': 1231632000, 'y': 5.26928858074074e-06}, {'x': 1231977600, 'y': 6.31320424296296e-06}, {'x': 1232323200, 'y': 5.91552208592593e-06}, {'x': 1232668800, 'y': 3.33058806518518e-06}, {'x': 1233014400, 'y': 5.12015777185185e-06}, {'x': 1233360000, 'y': 6.41262478222222e-06}, {'x': 1233705600, 'y': 6.11436316444444e-06}, {'x': 1234051200, 'y': 6.66117613037037e-06}, {'x': 1234396800, 'y': 5.8658118162963e-06}, {'x': 1234742400, 'y': 6.11436316444444e-06}, {'x': 1235088000, 'y': 6.36291451259259e-06}, {'x': 1235433600, 'y': 5.51783992888889e-06}, {'x': 1235779200, 'y': 5.07044750222222e-06}, {'x': 1236124800, 'y': 5.26928858074074e-06}, {'x': 1236470400, 'y': 5.56755019851852e-06}, {'x': 12368

## Get Company Financials


In [42]:
def safe_float(value, default=None):
    if value is None:
        return default
    if isinstance(value, (int, float)):
        if np.isnan(value) or np.isinf(value):
            return default
        return float(value)
    if isinstance(value, str):
        value = value.strip()
        if value.lower() in (
            "none",
            "",
            "-",
            "n/a",
            "null",
            "nan",
            "inf",
            "-inf",
            "infinity",
        ):
            return default
        try:
            result = float(value)
            if np.isnan(result) or np.isinf(result):
                return default
            return result
        except (ValueError, OverflowError):
            return default
    return default

In [46]:
def get_quaterly_data(ticker, api_key):
    base_url = "https://www.alphavantage.co/query"
    headers = {
        "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/jxl,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "user-agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36",
    }

    missing_info = []

    def track_value(raw_value, field_name, quarter):
        """Track missing or malformed values."""
        result = safe_float(raw_value)
        if result is None:
            missing_info.append(
                {
                    "ticker": ticker,
                    "quarter": quarter,
                    "field": field_name,
                    "raw_value": repr(raw_value),
                }
            )
        return result

    # 1. Get Income Statement Data
    params = {
        "function": "INCOME_STATEMENT",
        "symbol": ticker,
        "apikey": api_key,
    }

    res = requests.get(base_url, params=params, headers=headers, timeout=30)
    res.raise_for_status()
    data_income = res.json()

    # 2. Quaterly reports
    sleep(5)
    params = {
        "function": "BALANCE_SHEET",
        "symbol": ticker,
        "apikey": api_key,
    }

    res = requests.get(base_url, params=params, headers=headers, timeout=30)
    res.raise_for_status()
    data_balance = res.json()

    # 3. Monthly price data
    sleep(5)
    params = {
        "function": "TIME_SERIES_MONTHLY_ADJUSTED",
        "symbol": ticker,
        "apikey": api_key,
    }

    res = requests.get(base_url, params=params, headers=headers, timeout=30)
    res.raise_for_status()
    monthly_price = res.json()

    # 4. Income Calculations
    income_dict = {}
    for item in data_income.get("quarterlyReports", []):
        q = get_fiscal_quarter(item.get("fiscalDateEnding"))
        if q:
            income_dict[q] = item

    balance_dict = {}
    for item in data_balance.get("quarterlyReports", []):
        q = get_fiscal_quarter(item.get("fiscalDateEnding"))
        if q:
            balance_dict[q] = item

    price_dict = {}
    for date, item in monthly_price.get("Monthly Adjusted Time Series", {}).items():
        q = get_fiscal_quarter(date)
        if q:
            price_dict[q] = item

    variables = []
    for quarter, income_report in income_dict.items():
        balance_report = balance_dict.get(quarter)
        price_report = price_dict.get(quarter)

        # Log missing balance report
        if not balance_report:
            missing_info.append(
                {
                    "ticker": ticker,
                    "quarter": quarter,
                    "field": "balance_report",
                    "raw_value": "MISSING",
                }
            )
            continue

        # Extract base values with tracking
        net_income = track_value(income_report.get("netIncome"), "netIncome", quarter)
        total_assets = track_value(
            balance_report.get("totalAssets"), "totalAssets", quarter
        )

        # Try multiple fields for total_debt
        total_debt = safe_float(balance_report.get("shortLongTermDebtTotal"))
        debt_field_used = "shortLongTermDebtTotal"
        if total_debt is None:
            total_debt = safe_float(balance_report.get("longTermDebt"))
            debt_field_used = "longTermDebt"
        if total_debt is None:
            total_debt = safe_float(balance_report.get("totalDebt"))
            debt_field_used = "totalDebt"
        if total_debt is None:
            missing_info.append(
                {
                    "ticker": ticker,
                    "quarter": quarter,
                    "field": "total_debt (shortLongTermDebtTotal/longTermDebt/totalDebt)",
                    "raw_value": repr(
                        {
                            "shortLongTermDebtTotal": balance_report.get(
                                "shortLongTermDebtTotal"
                            ),
                            "longTermDebt": balance_report.get("longTermDebt"),
                            "totalDebt": balance_report.get("totalDebt"),
                        }
                    ),
                }
            )

        # Try multiple fields for cash
        cash = safe_float(balance_report.get("cashAndCashEquivalentsAtCarryingValue"))
        if cash is None:
            cash = safe_float(balance_report.get("cashAndShortTermInvestments"))
        if cash is None:
            cash = safe_float(balance_report.get("cash"))
        if cash is None:
            missing_info.append(
                {
                    "ticker": ticker,
                    "quarter": quarter,
                    "field": "cash (cashAndCashEquivalentsAtCarryingValue/cashAndShortTermInvestments/cash)",
                    "raw_value": repr(
                        {
                            "cashAndCashEquivalentsAtCarryingValue": balance_report.get(
                                "cashAndCashEquivalentsAtCarryingValue"
                            ),
                            "cashAndShortTermInvestments": balance_report.get(
                                "cashAndShortTermInvestments"
                            ),
                            "cash": balance_report.get("cash"),
                        }
                    ),
                }
            )

        shareholder_equity = track_value(
            balance_report.get("totalShareholderEquity"),
            "totalShareholderEquity",
            quarter,
        )
        shares_outstanding = track_value(
            balance_report.get("commonStockSharesOutstanding"),
            "commonStockSharesOutstanding",
            quarter,
        )

        if not price_report:
            missing_info.append(
                {
                    "ticker": ticker,
                    "quarter": quarter,
                    "field": "price_report",
                    "raw_value": "MISSING",
                }
            )
        stock_price = (
            track_value(
                price_report.get("5. adjusted close"), "5. adjusted close", quarter
            )
            if price_report
            else None
        )

        # Computed values with dependency checks
        roa = (
            (net_income / total_assets)
            if (net_income is not None and total_assets)
            else None
        )
        log_assets = (
            np.log(total_assets)
            if (total_assets is not None and total_assets > 0)
            else None
        )
        leverage = (
            (total_debt / total_assets)
            if (total_debt is not None and total_assets)
            else None
        )
        liquidity = (
            (cash / total_assets) if (cash is not None and total_assets) else None
        )
        market_cap = (
            (stock_price * shares_outstanding)
            if (stock_price is not None and shares_outstanding is not None)
            else None
        )
        pb_ratio = (
            (market_cap / shareholder_equity)
            if (market_cap is not None and shareholder_equity)
            else None
        )

        # BTC and hashrate data
        btc_price = btc_dict.get(quarter)
        if btc_price is None:
            missing_info.append(
                {
                    "ticker": ticker,
                    "quarter": quarter,
                    "field": "btc_price",
                    "raw_value": "MISSING from btc_dict",
                }
            )
        log_btc_price = (
            np.log(btc_price) if (btc_price is not None and btc_price > 0) else None
        )

        hashrate = hash_dict.get(quarter)
        if hashrate is None:
            missing_info.append(
                {
                    "ticker": ticker,
                    "quarter": quarter,
                    "field": "hashrate",
                    "raw_value": "MISSING from hash_dict",
                }
            )
        log_hashrate = (
            np.log(hashrate) if (hashrate is not None and hashrate > 0) else None
        )

        variables.append(
            {
                "ticker": ticker,
                "quarter": quarter,
                "net_income": net_income,
                "total_assets": total_assets,
                "total_debt": total_debt,
                "cash": cash,
                "shareholder_equity": shareholder_equity,
                "shares_outstanding": shares_outstanding,
                "stock_price": stock_price,
                "roa": roa,
                "log_assets": log_assets,
                "leverage": leverage,
                "liquidity": liquidity,
                "market_cap": market_cap,
                "pb_ratio": pb_ratio,
                "btc_price": btc_price,
                "log_btc_price": log_btc_price,
                "hashrate": hashrate,
                "log_hashrate": log_hashrate,
            }
        )

    df = pd.DataFrame(variables)
    return df, missing_info

In [47]:
api_keys = [AV1, AV2, AV3, AV4]

all_roa_dfs = []
missing_data_log = []

for i, ticker in enumerate(tickers):
    key_index = (i // 10) % len(api_keys)
    api_key = api_keys[key_index]
    print(f"Processing {ticker}...")
    roa_df, missing_info = get_quaterly_data(ticker, api_key)
    sleep(5)
    if roa_df is not None and not roa_df.empty:
        all_roa_dfs.append(roa_df)
    if missing_info:
        missing_data_log.extend(missing_info)

combined_df = (
    pd.concat(all_roa_dfs, ignore_index=True) if all_roa_dfs else pd.DataFrame()
)

# Save missing data log
missing_df = pd.DataFrame(missing_data_log)
if not missing_df.empty:
    missing_df.to_csv("missing_data_log.csv", index=False)
    print(f"Missing data logged: {len(missing_data_log)} entries")

Processing ABTS...
Processing ARLP...
Processing ABTC...
Processing APLD...
Processing ARBK...
Processing BTOG...
Processing BTDR...
Processing BITF...
Processing FUFU...
Processing BMNR...
Processing CAN...
Processing CANG...
Processing CIFR...
Processing CLSK...
Processing CORZ...
Processing DGXX...
Processing GREE...
Processing HUT...
Processing GPUS...
Processing IREN...
Processing LMFA...
Processing MARA...
Processing MIGI...
Processing RIOT...
Processing SAIH...
Processing SLAI...
Processing SLNH...
Processing SOS...
Processing ANY...
Processing WULF...
Processing OLB...
Processing NCTY...
Missing data logged: 1277 entries


/tmp/ipykernel_79321/2231932036.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pd.concat(all_roa_dfs, ignore_index=True) if all_roa_dfs else pd.DataFrame()


In [48]:
missing_df

,ticker,quarter,field,raw_value
0,ABTS,2025q1,total_debt (shortLongTermDebtTotal/longTermDeb...,"{'shortLongTermDebtTotal': 'None', 'longTermDe..."
1,ABTS,2024q3,total_debt (shortLongTermDebtTotal/longTermDeb...,"{'shortLongTermDebtTotal': 'None', 'longTermDe..."
2,ABTS,2024q2,total_debt (shortLongTermDebtTotal/longTermDeb...,"{'shortLongTermDebtTotal': 'None', 'longTermDe..."
3,ABTS,2024q1,total_debt (shortLongTermDebtTotal/longTermDeb...,"{'shortLongTermDebtTotal': 'None', 'longTermDe..."
4,ABTS,2023q4,total_debt (shortLongTermDebtTotal/longTermDeb...,"{'shortLongTermDebtTotal': 'None', 'longTermDe..."
...,...,...,...,...
1272,NCTY,2004q1,hashrate,MISSING from hash_dict
1273,NCTY,2003q4,balance_report,MISSING
1274,NCTY,2003q3,balance_report,MISSING
1275,NCTY,2003q2,balance_report,MISSING


In [49]:
combined_df

,ticker,quarter,net_income,total_assets,total_debt,cash,shareholder_equity,shares_outstanding,stock_price,roa,log_assets,leverage,liquidity,market_cap,pb_ratio,btc_price,log_btc_price,hashrate,log_hashrate
0,ABTS,2025q1,-447848.00,1.137130e+07,NaN,1.118929e+06,1.038095e+07,2369966.0,6.9600,-0.039384,16.246603,NaN,0.098399,1.649496e+07,1.588965,87187.77,11.375819,8.708571e+08,20.584988
1,ABTS,2024q3,-447848.00,1.137130e+07,NaN,1.118929e+06,1.038095e+07,2369966.0,8.4300,-0.039384,16.246603,NaN,0.098399,1.997881e+07,1.924565,65888.50,11.095719,6.811640e+08,20.339314
2,ABTS,2024q2,-6982.00,1.220201e+07,NaN,3.965830e+05,1.128131e+07,2370312.0,11.5500,-0.000572,16.317112,NaN,0.032501,2.737710e+07,2.426765,60315.67,11.007347,6.530446e+08,20.297156
3,ABTS,2024q1,-6982.00,1.220201e+07,NaN,3.965830e+05,1.128131e+07,2370312.0,16.2000,-0.000572,16.317112,NaN,0.032501,3.839905e+07,3.403775,70754.69,11.166974,5.950462e+08,20.204150
4,ABTS,2023q4,-9082174.00,1.231827e+07,NaN,8.841990e+05,1.131266e+07,2369966.0,12.9015,-0.737293,16.326594,NaN,0.071779,3.057612e+07,2.702823,43470.76,10.679844,4.331128e+08,19.886509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1387,NCTY,2005q1,12242105.00,1.026800e+09,29363748.0,7.935635e+08,8.653377e+08,47680.0,5457.5754,0.011923,20.749713,0.028597,0.772851,2.602172e+08,0.300712,NaN,NaN,NaN,NaN
1388,NCTY,2004q4,5489092.26,1.026800e+09,NaN,7.935635e+08,8.653377e+08,806208.0,NaN,0.005346,20.749713,NaN,0.772851,NaN,NaN,NaN,NaN,NaN,NaN
1389,NCTY,2004q3,5489092.00,NaN,NaN,NaN,NaN,731878.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1390,NCTY,2004q2,12242105.00,1.913600e+08,25073489.0,2.435843e+07,1.026093e+06,22700.0,NaN,0.063974,19.069667,0.131028,0.127291,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
c2 = combined_df.dropna()

In [ ]:
# Replace the last cell

c2_filtered = c2[
    pd.PeriodIndex(c2["quarter"].str.replace("q", "Q"), freq="Q").to_timestamp()
    >= pd.to_datetime("2009")
]

,ticker,quarter,net_income,total_assets,total_debt,cash,shareholder_equity,shares_outstanding,stock_price,roa,log_assets,leverage,liquidity,market_cap,pb_ratio,btc_price,log_btc_price,hashrate,log_hashrate
15,ABTS,2021q1,-1677049.0,3627303.0,91565.0,3627303.0,2306851.0,1079435.0,26.25,-0.462340,15.104000,0.025243,1.000000,2.833517e+07,12.283051,55783.71,10.929237,1.641284e+08,18.916159
16,ABTS,2020q4,-387160.0,2192192.0,382257.0,19402.0,91280.0,1289435.0,16.65,-0.176609,14.600413,0.174372,0.008851,2.146909e+07,235.200403,28856.59,10.270094,1.534821e+08,18.849094
17,ABTS,2020q3,78398.0,2295657.0,359549.0,5249.0,400773.0,1079435.0,13.35,0.034151,14.646530,0.156621,0.002286,1.441046e+07,35.956657,10840.80,9.291072,1.497813e+08,18.824687
18,ABTS,2020q2,-84511.0,630563.0,297219.0,441749.0,147999.0,1079435.0,10.20,-0.134025,13.354368,0.471355,0.700563,1.101024e+07,74.393996,9185.35,9.125365,1.153447e+08,18.563435
19,ABTS,2020q1,-147721.0,1495992.0,296510.0,8114.0,156633.0,1079435.0,28.35,-0.098745,14.218300,0.198203,0.005424,3.060198e+07,195.373786,5885.41,8.680232,1.016645e+08,18.437189
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,NCTY,2016q3,-118307891.0,350891994.0,232426295.0,38878076.0,-339616808.0,5534200.0,600.00,-0.337163,19.675989,0.662387,0.110798,3.320520e+09,-9.777255,606.79,6.408183,2.026558e+06,14.521849
1346,NCTY,2016q2,-440055996.0,396397533.0,225397462.0,21928254.0,-124269872.0,79456.0,702.00,-1.110138,19.797928,0.568615,0.055319,5.577811e+07,-0.448847,630.19,6.446021,1.530560e+06,14.241144
1347,NCTY,2016q1,-233858801.0,538094921.0,166909111.0,49010541.0,-197362396.0,77567.0,669.00,-0.434605,20.103546,0.310185,0.091082,5.189232e+07,-0.262929,413.74,6.025238,1.028362e+06,13.843477
1348,NCTY,2015q4,-384634060.0,538094921.0,166909111.0,49010541.0,-197362396.0,8964900.0,443.67,-0.714807,20.103546,0.310185,0.091082,3.977457e+09,-20.153065,433.23,6.071269,8.036470e+05,13.596915


In [60]:
c2_filtered.to_csv("./final_financials.csv")